# DataFrame Conversions
This notebook demonstrates the Resources conversion to pandas DataFrame and vice-versa.

## Initialisation

Run the [Blue Brain Nexus project creation notebook](https://github.com/BlueBrain/nexus-forge/blob/master/examples/notebooks/nexus-demo/00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/v1) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
# Get a KnowledgeGraphForge session
forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

## Imports

In [ ]:
import json

import pandas as pd
import numpy as np

from kgforge.core import Resource

In [ ]:
def pp(x): print(json.dumps(x, indent=4))

## List of Resources to DataFrame

In [ ]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [ ]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [ ]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [ ]:
persons = [jane, john]

In [ ]:
forge.register(persons)

In [ ]:
print(jane)

In [ ]:
print(john)

In [ ]:
pp(john._store_metadata)

In [ ]:
forge.as_dataframe(persons)

It is possible to specify what values should be replaced by `NaN` using the `na` parameter.

In [ ]:
forge.as_dataframe(persons, na="(missing)")

It is possible to specify a string to use in the column names to show nested values, the default is dot `.`

In [ ]:
forge.as_dataframe(persons, nesting="__")

Expanded parameter will show the each term with the json-ld context

In [ ]:
forge.as_dataframe(persons, expanded=True)

In [ ]:
forge.as_dataframe(persons, store_metadata=True)

## DataFrame to list of Resources

In [ ]:
data = pd.DataFrame([
    {
        "id": "33532569-70eb-4648-a7f1-f7ea22b0ce38",
        "type": "Person",
        "address.type": "PostalAddress",
        "address.country": "Switzerland",
        "address.locality": "Geneva",
        "email": "(missing)",
        "name": "Jane Doe",
    },
    {
        "id": "45e018f4-9ade-4ad0-bdcf-63902bf51cc1",
        "type": "Person",
        "address.type": np.nan,
        "address.country": np.nan,
        "address.locality": np.nan,
        "email": "john.smith@epfl.ch",
        "name": "John Smith",
    }
])

In [ ]:
data

In [ ]:
resources = forge.from_dataframe(data)

In [ ]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [ ]:
jane = Resource(id="33532569-70eb-4648-a7f1-f7ea22b0ce38", type="Person", name="Jane Doe", address=address, email="(missing)")

In [ ]:
john = Resource(id="45e018f4-9ade-4ad0-bdcf-63902bf51cc1", type="Person", name="John Smith", email="john.smith@epfl.ch")

In [ ]:
persons = [jane, john]

In [ ]:
resources == persons

In [ ]:
resources_na = forge.from_dataframe(data, na="(missing)")

In [ ]:
print(resources[0])

In [ ]:
print(resources_na[0])

In [ ]:
resources_nesting = forge.from_dataframe(data, nesting="__")

In [ ]:
print(resources[0])

In [ ]:
print(resources_nesting[0])